In [16]:
import pandas as pd
import numpy as np

In [17]:
anime_df = pd.read_csv('24_anime/anime.csv')
rating_df = pd.read_csv('24_anime/rating.csv')

In [ ]:
rating_df

In [18]:
## DROP RATING, NAME AND MEMBERS DUE TO INSIGNIFICANCE ##
anime_df = anime_df.drop(['rating','name','members'], axis=1)

## REMOVE -1 FROM RATINGS ##
rating_df = rating_df[rating_df.rating != -1]

(6337241, 3)

In [13]:
df_merge = pd.merge(rating_df, anime_df, on=['anime_id'])

In [14]:
df_merge

,user_id,anime_id,rating,genre,type,episodes
0,1,8074,10,"Action, Ecchi, Horror, Supernatural",TV,12
1,3,8074,6,"Action, Ecchi, Horror, Supernatural",TV,12
2,5,8074,2,"Action, Ecchi, Horror, Supernatural",TV,12
3,12,8074,6,"Action, Ecchi, Horror, Supernatural",TV,12
4,14,8074,6,"Action, Ecchi, Horror, Supernatural",TV,12
...,...,...,...,...,...,...
6337234,69964,23585,7,"Adventure, Drama, Slice of Life",Special,2
6337235,69964,33659,6,"Comedy, Magic, Slice of Life",Special,1
6337236,72800,30738,4,"Adventure, Kids",Movie,1
6337237,73135,8723,5,Comedy,OVA,4


In [19]:
df_merge.to_csv('data.csv', index=False)